Загрузите ваш граф. Я сохранил его в два JSON файла, но вы можете сделать это любым более удобным вам способом.
Я добавил дополнительное поле с полом. Можете выгрузить любые дополнительные поля по желанию. 

In [ ]:
import json
full_friends = json.load(open("nodes.txt", "tr"))
full_graph = json.load(open("edges.txt", "tr"))
import networkx as nx
G = nx.Graph()
for i in full_friends["items"]:
    G.add_node(i["id"], name = i["first_name"]+" "+i["last_name"], sex = i["sex"])
my_friends = list(nx.nodes(G))
for i in my_friends:
    if "items" in full_graph[str(i)]:
        for j in full_graph[str(i)]["items"]:
            if j in my_friends:
                G.add_edge(i, j)


Вариант отрисовки графа с дополнительными свойставми. Можно поиграть с разными параметрами. XKCD доваляет приятные мне эффекты, но не обязателен.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
def plot_graph(g):
    with plt.xkcd():
        k = nx.degree(g)
        plt.figure(1, figsize=(60,45))
        coord = nx.kamada_kawai_layout(g)
#        coord = nx.spring_layout(g)
#        coord = nx.circular_layout(g)
        labels={nd: g.node[nd]['name'] for (nd) in g.nodes()}
        nx.draw_networkx(g, pos=coord, nodelist=dict(k).keys(), node_size=[v*50 for v in dict(k).values()], 
                         font_size=17, node_color=list(dict(k).values()), labels=labels, cmap=plt.cm.Blues)

In [ ]:
plot_graph(G)

Поговорим о мерах центральности.

In [ ]:
deg = nx.degree(G)
sorted_by_degree = sorted(deg, key=lambda item: (-item[1], item[0]))

In [ ]:
for i in sorted_by_degree[:10]:
    print(nx.get_node_attributes(G, "name")[i[0]], i[1])

In [ ]:
subG = G.subgraph([n for n in G.neighbors(sorted_by_degree[0][0])])

In [ ]:
plot_graph(subG)

In [ ]:
nx.degree_centrality(G)

Поймите, как центральность по степени вершины соотносится со степенью вершины.

Рассмотрим определение центральности вершины, как взвешенную центральность соседей: $$x'_i = \sum_{j} a_{ij}x_j$$ Если мы будем итереироваться, то мы сойдемся к одному из собственных векторов. Тогда можно определить центральность как $$x_i = \frac{1}{\lambda} \sum_{j \in N(i)}x_t = \frac{1}{\lambda} \sum_{j \in G} a_{ij}x_j$$

Другое название такой меры -- степень влиятельности.

In [ ]:
ec = nx.eigenvector_centrality(G) 
sorted_by_ec = sorted(ec.items(), key=lambda item: (-item[1], item[0]))

In [ ]:
for i in sorted_by_ec[:20]:
    print(nx.get_node_attributes(G, "name")[i[0]], i[1])

In [ ]:
with plt.xkcd():
        k = ec
        plt.figure(1, figsize=(60,45))
        coord = nx.kamada_kawai_layout(G)
#        coord = nx.spring_layout(g)
#        coord = nx.circular_layout(g)
        labels={nd: G.node[nd]['name'] for (nd) in G.nodes()}
        nx.draw_networkx(G, pos=coord, nodelist=dict(k).keys(), node_size=[v*30000 for v in dict(k).values()], 
                         font_size=17, node_color=list(dict(k).values()), labels=labels, cmap=plt.cm.hot)

In [ ]:
Stange_directed_network = nx.DiGraph()
Stange_directed_network.add_edges_from([(1, 2), (1, 3), (1, 4), (2, 3), (2, 5), (3, 6), (5, 4)])
#nx.set_node_attributes(Stange_directed_network, [1, 2, 3, 4, 5, 6], "name")

nx.draw_networkx(Stange_directed_network)





In [ ]:
#nx.eigenvector_centrality(Stange_directed_network) 
nx.eigenvector_centrality(Stange_directed_network, max_iter=10000) 

Центральность по Кацу.
$$x_i = \sum_{k=1}^{\infty}\sum_{j=1}^N \alpha^k (A^k)_{ji}$$

где $\alpha$ дисконт за расстояние в диапазоне $(0,1)$.

Или в альтернативной форме: 

$$x_i = \alpha \sum_{j =1}^N a_{ij}(x_j+1).$$

Или немного в другой форме:
$$x_i = \alpha \sum_{j =1}^N a_{ij}x_j+\beta_i.$$

In [ ]:
nx.katz_centrality(Stange_directed_network, 0.9, max_iter = 1000) 

In [ ]:
kc = nx.katz_centrality(G, 0.02, max_iter = 1000) 
sorted_by_kc = sorted(kc.items(), key=lambda item: (-item[1], item[0]))

In [ ]:
for i in sorted_by_kc[:20]:
    print(nx.get_node_attributes(G, "name")[i[0]], i[1])

PageRank:
    $$x_i = \alpha \sum_{j } a_{ji}\frac{x_j}{d^{out}_j} + \frac{1-\alpha}{N}$$

In [ ]:
pr = nx.pagerank(G)
sorted_by_pr = sorted(pr.items(), key=lambda item: (-item[1], item[0]))
for i in sorted_by_pr[:15]:
    print(nx.get_node_attributes(G, "name")[i[0]], i[1])

Центральность по близости обратно пропорциональна среднему расстоянию: $$C(x)= \frac{1}{\sum_y d(y,x)}$$ А что будет в несвязном графе?  

In [ ]:
cc = nx.closeness_centrality(G) 
sorted_by_cc = sorted(cc.items(), key=lambda item: (-item[1], item[0]))
for i in sorted_by_cc[:20]:
    print(nx.get_node_attributes(G, "name")[i[0]], i[1])


In [ ]:
for i in sorted_by_cc[-30:]:
    print(nx.get_node_attributes(G, "name")[i[0]], i[1], deg[i[0]])


Центральность по посреднечеству характеризует долю кратчайших путей проходящих через данную вершину. Ожидается высокая у мостов. $$g(v)= \sum_{s \neq v \neq t}\frac{\sigma_{st}(v)}{\sigma_{st}},$$ где $\sigma_{st}$ -- число кратчайших путей между $s$ и $t$, $\sigma_{st}(v)$ -- число таких путей проходящих через $v$.

In [ ]:
bc = nx.betweenness_centrality(G) 
sorted_by_bc = sorted(bc.items(), key=lambda item: (-item[1], item[0]))
for i in sorted_by_bc[:20]:
    print(nx.get_node_attributes(G, "name")[i[0]], i[1])

Теперь, давайте одним глазком посмотрим немного в будущее. Поговорим о похожести.

In [ ]:
nx.degree_assortativity_coefficient(G)

In [ ]:
nx.attribute_assortativity_coefficient(G, "sex")

In [ ]:
sum([i==2 for i in nx.get_node_attributes(G, "sex").values()])

In [ ]:
sum([i==1 for i in nx.get_node_attributes(G, "sex").values()])

In [ ]:
sex_variation = {}
sex = nx.get_node_attributes(G, "sex")
for pers in G.nodes():
    ngb = list(G.neighbors(pers))
    sex_variation[pers] = list([sum([sex[i]==1 for i in ngb]),sum([sex[i]==2 for i in ngb])]) 
    

In [ ]:
a,b = 0.1,0.1 
sorted_by_sex_pref = sorted(sex_variation.items(), key=lambda item: (-abs((item[1][0]+a)/(item[1][0]+item[1][1]+a+b)-0.5), item[0]))
for i in sorted_by_sex_pref[:20]:
    print(nx.get_node_attributes(G, "name")[i[0]], i[1])

In [ ]:
(sum(sex.values())-len(sex))/len(sex)

In [ ]:
l = [list(G.neighbors(i[0])) for i in G.nodes(data='name') if i[1]=="Анастасия Шишкова"]
[i for i in G.nodes(data='name') if i[0] in l[0]]